## Pybullet tutorial
by SeoYoung Son
##### Reference
* <a href = "https://m.blog.naver.com/PostView.nhnblogId=einsbon&logNo=221294095704&proxyReferer=https%3A%2F%2Fwww.google.com%2F">네이버 블로그: 미래 가젯 연구소</a>
* <a href = "https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.2ye70wns7io3">Pybullet Quickstart Guide</a>
* <a href = "https://medium.com/@gerardmaggiolino/creating-openai-gym-environments-with-pybullet-part-1-13895a622b24">Creating OpenAI Gym Environments with Pybullet</a>
* <a href = "http://wiki.ros.org/urdf/Tutorials">URDF tutorials</a>

#### 이번 시간 학습 목표:
* Pybullet에 대해 간단히 알아보는 시간을 갖는다
* ROS에 들어가는 모델인 URDF를 작성해본다.   

본 문서는 SAI 스터디용 자료이므로 허락없이 퍼가시는 것을 금합니다

### Getting started..
이 문서는 어느 정도의 파이썬 지식과 기초적인 딥러닝 지식을 보유한 사람들을 위해 작성하였습니다.  
아래 준비물들을 준비하심 좋을 것 같습니다.  
* Pycharm (2018.02 이상)
* Python 3.7 ver.
* Notepad++ 혹은 VisualCode

cmd 및 터미널을 열어서 아래 모듈들을 설치해주세요!
```cmd
pip install pybullet
pip install tensorflow         //필요하면 설치
pip install baselines          //현재 tensorflow2.0이 반영이 안되어 RL의 deepQ 기능 사용 불가
```

이제 준비를 모두 다 끝마치셨다면 아래 코드를 복사해서 잘 돌아가는지 확인해봅시다.  
각 코드가 무엇을 의미하는지 차근차근 설명해드리겠습니다.

In [2]:
import pybullet as p
import time
import pybullet_data

In [ ]:
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.8)
p.setTimeStep(1/200)

planeID = p.loadURDF("plane.urdf")
cubeStartPos = [2, -1, 1]
cubeStartOrientation = p.getQuaternionFromEuler([0, 0, 3.14])
robotID = p.loadURDF("r2d2.urdf", cubeStartPos, cubeStartOrientation)

for i in range(10000):
    p.stepSimulation()
    time.sleep(1 / 200)
p.disconnect(physicsClient)

### URDF 작성법

이번에는 URDF 작성하는 방법에 대해 알아보겠습니다.  
URDF는 Unified Robotic Description Format 의 약어로 ROS에 사용되는 xml 파일 포맷입니다.  
이외 Pybullet에서는 SDF와 같은 기타 포맷도 사용되지만 이번 시간에는 URDF에 대해서만 간략하게 알아보겠습니다. 

간단한 로봇 팔을 손으로 직접 짜보는 시간을 가질 것입니다.

<center><em>For more information visit:</em></center>  

[![GitHub](http://img.shields.io/badge/-Tech%20blog-black?style=flat-square&logo=github&link=https://github.com/ameliacode)](https://github.com/ameliacode)